In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns',None)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

import matplotlib.pyplot as plt 

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns 
from tqdm import tqdm
tqdm.pandas()

import datetime
import ast
import math
import gc

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=7, progress_bar=True)

from sklearn.preprocessing import MinMaxScaler

def say(comment):
  os.system(f'say "{comment}"')

# warming 제거
import warnings
warnings.filterwarnings('ignore')
from branca.colormap import linear

# notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것
%matplotlib inline       

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df = pd.read_pickle('0216_0357_skill2pgm_formodel_ver1.pkl')

In [3]:
df['60_norm'] = df['인입콜'] / df['duration'] * 60

In [4]:
df.describe()

,인입콜,year,duration,판매가평균,price_max_all,price_min_all,prod_num,한정표현_num,주말/공휴일,sh_num,month_sin,month_cos,hour_sin,hour_cos,시차변수,week,price_max_top3,price_min_top3,price1,price2,price3,score1,score2,score3,60_norm
count,34051.000000,34051.000000,34051.000000,3.405100e+04,3.405100e+04,3.405100e+04,34051.000000,34051.000000,34051.000000,30949.000000,3.405100e+04,3.405100e+04,3.405100e+04,3.405100e+04,33736.000000,34051.000000,3.405100e+04,3.405100e+04,3.405100e+04,3.405100e+04,3.405100e+04,34051.000000,34051.000000,34051.000000,34051.000000
mean,540.260374,1.523421,60.605650,9.978643e+05,1.106457e+06,9.209210e+05,2.326833,0.347684,0.183196,1.740315,-1.344158e-02,4.026106e-03,3.459380e-02,-1.774256e-02,7646.900018,27.055828,1.042208e+06,9.440166e+05,9.910973e+05,9.892963e+05,9.897475e+05,0.048288,0.047106,0.045639,464.936890
std,866.401881,0.499459,26.819445,3.674586e+06,3.726481e+06,3.663482e+06,3.772354,0.533538,0.386833,0.453610,7.044509e-01,7.096347e-01,7.260723e-01,6.865397e-01,2040.914712,15.089268,3.701406e+06,3.677125e+06,3.687755e+06,3.687387e+06,3.687929e+06,0.052206,0.050315,0.047085,580.352054
min,0.000000,1.000000,10.000000,1.690000e+04,1.690000e+04,9.000000e+03,1.000000,0.000000,0.000000,1.000000,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,21.600000,1.000000,1.500000e+04,9.000000e+03,9.900000e+03,1.100000e+04,9.000000e+03,0.000000,0.000000,0.000000,0.000000
25%,80.000000,1.000000,60.000000,6.242500e+04,6.900000e+04,5.980000e+04,1.000000,0.000000,0.000000,1.000000,-8.660254e-01,-5.000000e-01,-7.071068e-01,-7.071068e-01,6196.400000,14.000000,6.490000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,0.006587,0.006587,0.006587,90.000000
50%,189.000000,2.000000,60.000000,1.280000e+05,1.480000e+05,9.900000e+04,1.000000,0.000000,0.000000,2.000000,-2.449294e-16,6.123234e-17,1.224647e-16,-1.836970e-16,7398.200000,27.000000,1.340000e+05,9.900000e+04,1.190000e+05,1.190000e+05,1.180000e+05,0.033105,0.033105,0.033105,201.428571
75%,797.000000,2.000000,60.000000,7.590000e+05,8.658000e+05,3.990000e+05,2.000000,1.000000,0.000000,2.000000,5.000000e-01,8.660254e-01,7.071068e-01,7.071068e-01,8936.600000,40.000000,7.290000e+05,4.895000e+05,6.570000e+05,6.490000e+05,6.800000e+05,0.052381,0.052381,0.047506,744.428571
max,75228.000000,2.000000,240.000000,6.897500e+07,6.897500e+07,6.897500e+07,66.000000,4.000000,1.000000,5.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,16647.600000,53.000000,6.897500e+07,6.897500e+07,6.897500e+07,6.897500e+07,6.897500e+07,0.366812,0.366812,0.366812,36109.440000


In [10]:
new_df = df[df['60_norm']<2000].drop('60_norm',axis=1)

In [12]:
new_df.describe()

,인입콜,year,duration,판매가평균,price_max_all,price_min_all,prod_num,한정표현_num,주말/공휴일,sh_num,month_sin,month_cos,hour_sin,hour_cos,시차변수,week,price_max_top3,price_min_top3,price1,price2,price3,score1,score2,score3
count,33545.000000,33545.000000,33545.000000,3.354500e+04,3.354500e+04,3.354500e+04,33545.000000,33545.000000,33545.000000,30445.000000,3.354500e+04,3.354500e+04,3.354500e+04,3.354500e+04,33236.000000,33545.000000,3.354500e+04,3.354500e+04,3.354500e+04,3.354500e+04,3.354500e+04,33545.000000,33545.000000,33545.000000
mean,498.266895,1.527441,60.278253,1.011210e+06,1.120053e+06,9.337730e+05,2.311432,0.341511,0.183872,1.738315,-1.469249e-02,2.255408e-03,3.380154e-02,-1.064658e-02,7636.668745,27.066865,1.056004e+06,9.570916e+05,1.004567e+06,1.002755e+06,1.003143e+06,0.047272,0.046180,0.045001
std,679.839280,0.499254,26.586659,3.700530e+06,3.752375e+06,3.689488e+06,3.763090,0.529638,0.387386,0.453173,7.046257e-01,7.094445e-01,7.256974e-01,6.871224e-01,2035.036911,15.063934,3.727448e+06,3.703181e+06,3.713791e+06,3.713425e+06,3.713974e+06,0.050810,0.049044,0.046731
min,0.000000,1.000000,10.000000,1.690000e+04,1.690000e+04,9.000000e+03,1.000000,0.000000,0.000000,1.000000,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,21.600000,1.000000,1.500000e+04,9.000000e+03,9.900000e+03,1.100000e+04,9.000000e+03,0.000000,0.000000,0.000000
25%,78.000000,1.000000,55.000000,6.400000e+04,6.900000e+04,5.980000e+04,1.000000,0.000000,0.000000,1.000000,-8.660254e-01,-5.000000e-01,-7.071068e-01,-7.071068e-01,6196.400000,14.000000,6.490000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,0.006587,0.006587,0.006587
50%,184.000000,2.000000,60.000000,1.290000e+05,1.490000e+05,9.900000e+04,1.000000,0.000000,0.000000,2.000000,-2.449294e-16,-1.836970e-16,1.224647e-16,6.123234e-17,7380.400000,27.000000,1.380000e+05,1.080000e+05,1.199000e+05,1.190000e+05,1.190000e+05,0.033105,0.033105,0.033105
75%,746.000000,2.000000,60.000000,7.800000e+05,8.690000e+05,4.590000e+05,2.000000,1.000000,0.000000,2.000000,5.000000e-01,8.660254e-01,7.071068e-01,7.071068e-01,8895.200000,40.000000,7.590000e+05,5.090000e+05,6.990000e+05,6.990000e+05,6.990000e+05,0.052381,0.047506,0.047506
max,6342.000000,2.000000,240.000000,6.897500e+07,6.897500e+07,6.897500e+07,66.000000,4.000000,1.000000,5.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,16647.600000,53.000000,6.897500e+07,6.897500e+07,6.897500e+07,6.897500e+07,6.897500e+07,0.366812,0.366812,0.366812
